# League Analysis and Graphing (LAG)

[nbviewer link to render graphs](https://nbviewer.jupyter.org/github/joeeoj/fantasy-football/blob/main/lag.ipynb)

## League rules

* 12 teams
* 0.5 PPR
* 1 QB, 2 RB, 3 WR, 1 TE, 1 FLEX (RB/WR/TE), 1 DST, 1 K
* auction draft - \$200
* waivers - \$100 (Free-Agent Acquisition Budget / FAAB)

## LAG analysis



In [1]:
from pathlib import Path
from typing import List

import altair as alt
import pandas as pd

pd.set_option('max_columns', 100)

In [2]:
DATA_DIR = Path.cwd() / 'data'

POINTS_COL = 'total_points'

In [3]:
df = pd.read_csv(DATA_DIR / 'league_data_2020.csv')

# don't need this in our league
df = df[df['position'] != 'DT']

df['points'] = df[POINTS_COL]
df = df.drop(['total_points', 'projected_total_points'], axis=1)

df = df.sort_values(['position', 'rank'], ascending=[False, True])

df.head()

,id,name,position,team,rank,points
78,16800,Davante Adams,WR,GB,1,300.9
111,3116406,Tyreek Hill,WR,KC,2,285.4
113,2976212,Stefon Diggs,WR,BUF,3,265.1
175,3925357,Calvin Ridley,WR,ATL,4,236.5
49,15795,DeAndre Hopkins,WR,ARI,5,230.3


In [4]:
m = (df['rank'] > 0) & (df['rank'] < 21)
pos_points_ranked = (pd.pivot_table(df[m], index='rank', columns='position',
                                    values='points', aggfunc='sum')
                       .astype(int))

pos_points_ranked

position,D/ST,K,QB,RB,TE,WR
rank,,,,,,
1,174,169,395,336,260,300
2,164,166,382,323,225,285
3,157,153,378,315,150,265
4,146,151,374,237,141,236
5,145,146,369,235,141,230
6,137,145,359,234,140,230
7,127,142,343,225,132,229
8,127,142,337,214,126,217
9,117,138,332,199,121,215


In [5]:
pos_sort = ['QB', 'RB', 'WR', 'TE', 'K', 'D/ST']

(alt.Chart(df)
    .mark_line()
    .encode(x='rank', y='points', color=alt.Color('position', sort=pos_sort),
            tooltip=['points', 'name', 'position'])
    .transform_filter((alt.datum.rank > 0) & (alt.datum.rank < 21))
    .properties(width=600, height=400)
    .interactive())

alt.Chart(...)

### All QBs

In [6]:
(alt.Chart(df)
    .mark_point()
    .encode(x='rank', y='points', tooltip=['name', 'points', 'rank'])
    .transform_filter((alt.datum.position == 'QB'))
    .properties(width=600, height=400)
    .interactive())

alt.Chart(...)

## 3-10-20

Compare the decline in score by position by averaging semi-arbitrary groups of ranks: 1-3, 4-10, and 11-20.

In [7]:
three = df[(df['rank'] > 0) & (df['rank'] <= 3)].groupby('position')['points'].mean().reset_index()
three['label'] = 'top_3_avg'

ten = df[(df['rank'] > 3) & (df['rank'] <= 10)].groupby('position')['points'].mean().reset_index()
ten['label'] = '4_to_10_avg'

twenty = df[(df['rank'] > 10) & (df['rank'] <= 20)].groupby('position')['points'].mean().reset_index()
twenty['label'] = '11_to_20_avg'

three_ten_twenty = pd.pivot_table(pd.concat([three, ten, twenty]), index='position', columns='label', values='points', aggfunc='sum').astype(int)

three_ten_twenty['drop_3_to_10'] = three_ten_twenty['top_3_avg'] - three_ten_twenty['4_to_10_avg']
three_ten_twenty['drop_11_to_20'] = three_ten_twenty['4_to_10_avg'] - three_ten_twenty['11_to_20_avg']
three_ten_twenty['drop_3_to_20'] = three_ten_twenty['top_3_avg'] - three_ten_twenty['11_to_20_avg']

# figure 3.3 ordering
three_ten_twenty = three_ten_twenty[['top_3_avg', '4_to_10_avg', 'drop_3_to_10', '11_to_20_avg', 'drop_11_to_20', 'drop_3_to_20']]
three_ten_twenty = three_ten_twenty.reindex(['QB', 'RB', 'WR', 'TE', 'K', 'D/ST'])

three_ten_twenty

label,top_3_avg,4_to_10_avg,drop_3_to_10,11_to_20_avg,drop_11_to_20,drop_3_to_20
position,,,,,,
QB,385,350,35,261,89,124
RB,325,221,104,175,46,150
WR,283,224,59,194,30,89
TE,211,131,80,110,21,101
K,162,143,19,124,19,38
D/ST,165,130,35,103,27,62


### point decline by tier

In [8]:
drop_cols_only = three_ten_twenty[[c for c in three_ten_twenty.columns if c.startswith('drop')]]
drop_cols_only

label,drop_3_to_10,drop_11_to_20,drop_3_to_20
position,,,
QB,35,89,124
RB,104,46,150
WR,59,30,89
TE,80,21,101
K,19,19,38
D/ST,35,27,62


In [9]:
def pos_avg_line_chart(t: pd.DataFrame, label_sort: List[str] = None) -> alt.vegalite.v4.api.Chart:
    return (alt.Chart(t)
               .mark_line(point={'size': 50})
               .encode(x=alt.X('label', sort=label_sort), y='value', color='position', tooltip=['position', 'value'])
               .properties(width=600, height=400)
               .interactive())

In [10]:
t = drop_cols_only.reset_index().melt(id_vars=['position'], value_vars=drop_cols_only.columns)
label_sort = ['drop_3_to_10', 'drop_11_to_20', 'drop_3_to_20']

pos_avg_line_chart(t, label_sort)

alt.Chart(...)

### tiers instead of point declines

In [11]:
avgs_only = three_ten_twenty[[c for c in three_ten_twenty.columns if not c.startswith('drop')]]
t = avgs_only.reset_index().melt(id_vars=['position'], value_vars=avgs_only.columns)

pos_avg_line_chart(t)

alt.Chart(...)